# TMDB Movie Analysis

Author: Hilton Lam (2024)

By leveraging their public API, I use datasets from TMDB, a popular movie and TV review platform, to explore key trends in the film industry. Using Python, I clean and preprocess the data before analysing patterns such as production volume and popularity by country, genre-based popularity, and the relationship between budget and popularity. Additionally, I leverage the **Gemini LLM API** to identify common themes among top-performing movies.

### Notebook 1: Data Collection from the TMDB database

In this notebook, I begin by collecting data using the TMDB API [following the documentation here](https://developer.themoviedb.org/reference/intro/getting-started). The dataset collected in this notebook is subsequently used in notebook 2 for analysis, which can be found in the same folder.

In [1]:
import urllib.request
import json
import pandas as pd

# Insert your API key for TMDB
api_key = 'insert_your_api_key_here'

### Dataset 1: Pull all the now playing movies

This step can take a minute to load. It will pull a list of movies that are now playing.

In [8]:
df_movies_now_playing = pd.DataFrame() # create an empty dataframe to append the data pulled

# In this API end point, we must specify the page number as each API call only pulls data from one page. Hence, I created a loop to pull data from many pages and append the data to a dataframe.

page_num = 1 # Start pulling the data from page 1

while True:
    url = f"https://api.themoviedb.org/3/movie/now_playing?api_key={api_key}&language=en-US&page={page_num}"
    response = urllib.request.urlopen(url)
    raw_json = response.read().decode("utf-8")
    response_data = {}
    response_data = json.loads(raw_json)
    df = pd.DataFrame(response_data['results']) # extract the component of the JSON and convert to a dataframe
    # Append to the dataframe if there is content (at least one row) in the result
    if len(df) > 0: # if the API call returns any result
        df_movies_now_playing = pd.concat([df_movies_now_playing,df], ignore_index = True) 
        page_num += 1
    else: # there is no more content, then end the loop
        break

In [10]:
df_movies_now_playing.shape # Get to know the dataset

(5731, 14)

In [12]:
df_movies_now_playing.head() # See the dataset

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/417tYZ4XUyJrtyZXj7HpvWf1E8f.jpg,"[16, 878, 10751]",1184918,en,The Wild Robot,"After a shipwreck, an intelligent robot called...",4950.682,/wTnV3PCVW5O92JMrFvvrRcV39RU.jpg,2024-09-12,The Wild Robot,False,8.600,1442
1,False,/3V4kLQg0kSqPLctI5ziYWabAZYF.jpg,"[28, 878, 12]",912649,en,Venom: The Last Dance,Eddie and Venom are on the run. Hunted by both...,2994.212,/k42Owka8v91trK1qMYwCQCNwJKr.jpg,2024-10-22,Venom: The Last Dance,False,6.672,67
2,False,/7h6TqPB3ESmjuVbxCxAeB1c9OB1.jpg,"[27, 18, 878]",933260,en,The Substance,A fading celebrity decides to use a black mark...,2641.055,/lqoMzCcZYEFK729d6qzt349fB4o.jpg,2024-09-07,The Substance,False,7.288,864
3,False,/4KHEK6AQFHhv4TDtL3KLReePB05.jpg,"[28, 27, 53]",1029235,en,Azrael,"In a world where no one speaks, a devout femal...",1151.735,/qpdFKDvJS7oLKTcBLXOaMwUESbs.jpg,2024-09-27,Azrael,False,6.089,90
4,False,/eQEgKIRF7KeVGBQ8IYcklpU8RPf.jpg,"[27, 53]",1034541,en,Terrifier 3,Five years after surviving Art the Clown's Hal...,1760.315,/63xYQj1BwRFielxsBDXvHIJyXVm.jpg,2024-10-09,Terrifier 3,False,7.163,135


In [15]:
# Exploratory analysis revealed that there are duplicate rows in the dataset
# df_movies_now_playing['id'].count()
# df_movies_now_playing['id'].nunique()
# df_movies_now_playing[(df_movies_now_playing['id'].duplicated())]

# Drop duplicate rows based on movie id
df_movies_now_playing = df_movies_now_playing.drop_duplicates(subset=['id'], keep='first')

# Creating a list of now-playing movie ids
list_movies_now_playing = df_movies_now_playing['id'].tolist()

In [17]:
len(list_movies_now_playing) # No. of unique movies now playing

4732

### Dataset 2: Pull movie details for the now playing movies

Pull more details about each now playing movies, such as country, tagline, languages, budget, etc.

Note: this step will take minutes to load. This step will also produce a file "movie_details_backup.csv" , which can be used for later analyses or as a backup.

In [26]:
df_movie_details = pd.DataFrame() # create an empty dataframe to append the data pulled

n = 0 # Start a counter which helps me debug when someone is wrong in the loop

for movie_id in list_movies_now_playing: # pull details of each movie one by one
    try:
        url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}"
        response = urllib.request.urlopen(url)
        raw_json = response.read().decode("utf-8")
        response_data = {}
        response_data = json.loads(raw_json)
        df = pd.json_normalize(response_data) #flatten the JSON file and put in a dataframe
        df_movie_details = pd.concat([df_movie_details,df], ignore_index = True) # Append to the dataframe
        # print (f'successfully pulled details from movie {n}, {movie_id}') # used in validation
        n += 1
        if n % 500 == 0:
            df_movie_details.to_csv("movie_details_backup.csv", index=False) # save the dataframe in a csv file for every 500 rows for backup
    except urllib.error.HTTPError as e: # previously ran into 404 error, wrote this for error handling
        if e.code == 404:
            print(f"Movie ID {movie_id} not found (404), skipping")
        else:
            print(f"HTTP error for movie ID {movie_id}: {e}")
    except Exception as e:
        print(f"An error occurred for movie ID {movie_id}: {e}")

df_movie_details.to_csv("movie_details_backup.csv", index=False) # save the final dataframe into a csv file for easy access later due to long load time

In [ ]:
# Use this step to retrieve the saved file if you do not want to pull the data from the API again
# df_movie_details = pd.read_csv('movie_details_backup.csv')

In [30]:
# Exploratory analysis shows there are no duplicate records
# df_movie_details['id'].count()
# df_movie_details['id'].nunique()

df_movie_details.head()

,adult,backdrop_path,budget,genres,homepage,id,imdb_id,origin_country,original_language,original_title,...,tagline,title,video,vote_average,vote_count,belongs_to_collection.id,belongs_to_collection.name,belongs_to_collection.poster_path,belongs_to_collection.backdrop_path,belongs_to_collection
0,False,/417tYZ4XUyJrtyZXj7HpvWf1E8f.jpg,78000000,"[{'id': 16, 'name': 'Animation'}, {'id': 878, ...",https://www.thewildrobotmovie.com,1184918,tt29623480,[US],en,The Wild Robot,...,Discover your true nature.,The Wild Robot,False,8.600,1444,1370345.0,The Wild Robot Collection,/cioNnsPSHJH9gsUSETPFHh0m6MT.jpg,/f6G8QPeod5ngQMs5Fe1O4LdphB7.jpg,NaN
1,False,/3V4kLQg0kSqPLctI5ziYWabAZYF.jpg,110000000,"[{'id': 28, 'name': 'Action'}, {'id': 878, 'na...",https://venom.movie,912649,tt16366836,[US],en,Venom: The Last Dance,...,'Til death do they part.,Venom: The Last Dance,False,6.493,75,558216.0,Venom Collection,/hoTLlTIohrzQ13HQVkZrDlvffuT.jpg,/vq340s8DxA5Q209FT8PHA6CXYOx.jpg,NaN
2,False,/7h6TqPB3ESmjuVbxCxAeB1c9OB1.jpg,17500000,"[{'id': 27, 'name': 'Horror'}, {'id': 18, 'nam...",https://www.the-match-factory.com/catalogue/fi...,933260,tt17526714,[GB],en,The Substance,...,"If you follow the instructions, what could go ...",The Substance,False,7.300,867,NaN,NaN,NaN,NaN,NaN
3,False,/4KHEK6AQFHhv4TDtL3KLReePB05.jpg,12000000,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",,1029235,tt22173666,[US],en,Azrael,...,Many years after the rapture… Among the surviv...,Azrael,False,6.111,90,NaN,NaN,NaN,NaN,NaN
4,False,/eQEgKIRF7KeVGBQ8IYcklpU8RPf.jpg,2000000,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",https://terrifier3.com/,1034541,tt27911000,[US],en,Terrifier 3,...,"You better not shout, you better not cry.",Terrifier 3,False,7.200,140,727761.0,Terrifier Collection,/4xIzrMcEvCzJm5qAl92WMHLSIeM.jpg,/zREjCmCHIHdEF6ufPoDQjhl4Wdm.jpg,NaN


This output data file will be used for analysis in the second workbook